# Topic Modeling with Gensim¶
1. 소개
자연언어 처리의 주요 어플리케이션중 하나는 커다란 볼륨의 텍스트로부터 사람들이 어떤 주제에 대해서 이야기하는지 자동으로 토픽을 추출하는 것입니다. 큰 텍스트의 예로는 소셜 미디어의 피드, 호텔, 영화등의 고객 리부, 사용자 피드백, 뉴스기사, 고객불만사항 전자메일 등이 있습니다. 사람들이 무엇을 원하는가를 알고 그들의 문제와 의견을 이해하는것은 사업이나, 관리, 정치 캠페인에 유용합니다. 그리고 그렇게 많은 양을 직접 읽고 주제를 이해하는 것은 정말 어렵습니다. 따라서 텍스트를 읽고 논의된 주제를 출력할 수 있는 자동화된 알고리즘이 필요합니다. 이 튜토리얼에서는 ’20개의 뉴스그룹’ 데이터세트의 실제 예제를 사용하고 LDA를 사용하여 있는 그대로의 논의된 주제를 추출해낼 것입니다. Gensim 패키지에 있는 Latent Dirichlet Allocation(LDA)를 사용할 것이고, (Gensim을 통해) Mallet’s의 구현도 함께 사용할 것입니다. Mallet은 LDA를 효율적으로 구현하였습니다. 더 빠르게 수행되고 더 좋은 토픽을 제시해주는 것으로 알려져있습니다. 우리는 또한 각 토픽의 양과 기여도 비율을 추출하여 얼마나 중요한 주제인지를 알아보도록 하겠습니다.

시작해봅시다.


### 참고사이트

 http://www.engear.net/wp/topic-modeling-gensimpython/ 
 
 https://github.com/smilemango/lda_study/blob/master/lda_with_gensim.ipynb


# 2. 사전 준비 – nltk stopwords와 spacy 모델 다운받기
텍스트 전처리를 위하여 NLTK의 stopwords와 spacy의 en 모델이 필요합니다. 나중에 우리는 표준형변환(lemmatization)을 위하여 spacy 모델을 사용할 것입니다. 표준형변환(Lemmatization)은 근원 단어로 변환하는 것입니다. 예를들어 ‘machines’의 lemma 단어는 ‘machine’입니다. 마찬가지로, ‘walking’ -> ‘walk’,’mice’ -> ‘mouse’ 등입니다.

In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Run in terminal or command prompt 
python3 -m spacy download en

# 3. Import Packages
이 튜토리얼에서 사용할 핵심 패키지는 re , gensim , spacy , pyLDAvis  입니다. 이외에 우리는 matplotlib , numpy , pandas 를 사용하여 데이터를 다루고 시각화 할것입니다. 이것들을 import 합시다.

In [2]:
import re 
import numpy as np
import pandas as pd
from pprint import pprint

#Gensism
import gensim
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy for lemmatization 
#spaCy는 파이썬의 자연어처리를 위한 오픈 소스 기반 라이브러리다.
#아직 한국어는 지원하지 않지만, 텍스트 전처리에서 좋은 성능을 보여줌
import spacy

#Plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt 
%matplotlib inline

#Enable logging for gensim -optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 4. LDA는 무엇을 합니까?
토픽 모델링에 대한 LDA의 접근방식은 각 도큐먼트를 특정 비율의 토픽의 집합으로 가정합니다. 그리고 다시, 각 토픽은 다시 각 키워드의 일정 비율로 구성됩니다.알고리즘에 토픽의 개수를 입력하면, 문서 내에서의 토픽 분포화 토픽 안에서의 키워드 분포를 재정렬하여 토픽-키워드 분포의 적절한 구성을 얻을 수 있습니다. 토픽을 말할때, 실제로 그리고 어떻게 표현되는가? 토픽은 일반적으로 표현되는 지배적인 키워드의 모음입니다. 키워드를 살펴보는것만으로 주제가 무엇인지를 파악할 수 있습니다.

좋은 토픽 구분을 얻기 위한 핵심 요소는 다음과 같습니다.

1. 처리된 텍스트 데이터의 품질
2. 텍스트가 말하고자하는 토픽의 다양성
3. 토픽 모델링 알고리즘의 선택
4. 알고리즘에 공급된 주제수
5. 알고리즘 튜닝 파라미터
 

## 5. Stopwords(불용단어) 준비하기
앞서 우리는 stopwords를 다운로드 하였습니다. 이것을 import 하여 사용하도록 합시다.

In [4]:
#NLTK Stop words
from nltk.corpus import stopwords 
stop_words=stopwords.words('english')
stopwords.expend(['from','subject','re','edu','use'])

AttributeError: 'WordListCorpusReader' object has no attribute 'expend'

## 6. 뉴스그룹 데이터 임포트하기
본 실습을 위하여 ’20개 뉴스그룹’ 데이터를 사용할 것입니다. 이 데이터에는 20개의 서로 다른 주제로 구성된 약 11,000개의 뉴스그룹 게시물이 포함되어 있습니다. 이것은 newsgroups.json 에서 다운로드 가능합니다.

이것을 https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json  'pandas.read_json' 을 통해서 읽으며, 결과 데이터 세트는 보여지는 것 처럼 3개의 컬럼을 가지고 있습니다.

In [5]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## 7. 이메일 주소와 줄바꿈 문자 제거
보는 것처럼 이메일, 줄바꿈 문자 및 불필요한 공백이 있어서 불편합니다. 이를 정규식을 사용하여 제거하도록 합시다.

In [6]:
# Convert to list
data = df.content.values.tolist()
 
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
 
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
 
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
 
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


전자메일과 불필요한 공백을 제거하고나서도 여전히 지저분해보입니다. LDA에서 처리하기에는 부적합합니다. 토큰화를 통해 각 문장을 단어의 list로 분해하면서 처리 과정에서 지저분한 모든 문자를 지워야합니다.

이를 위해서 Gensim의 simple_process  가 적절합니다.

## 8. 단어 토큰화와 텍스트 클린업
각 문장의 구두점과 불필요한 문자를 제거하여, 단어의 list로 토크나이징합시다. 이작업에는 Gensim의 simple_preprocess() 가 훌륭합니다. 추가적으로 나는 구두점을 제거하기 위해서 deacc=True 로 설정하였습니다.

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) #deacc=True removes punctions
data_words=list(sent_to_words(data))
print(data_words[:1])   

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## 9. Bigram과 Trigram 모델 만들기
Bigram은 문서에서 자주 발생되는 2연 단어입니다. Tigram은 자주 발생하는 3연 단어입니다. 이 예에서는 ‘front_bumper’,’oil_leak’,’maryland_college_park’등이 있습니다. Gensim의 Phrases 모델은 bigram, trigram, quadgram 등 그 이상을 구현할 수 있습니다. Phrases에 대한 두가지 중요한 인수는 min_count및 임계값입니다. 이 매개변수의 값이 높을수록 단어가 바이그램으로 결합되는 것이 어렵습니다.

In [8]:
#Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words,min_count=5,threshold=100) #higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

#Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


## 바이그램 모형이란?
만약 단어의 활용이 바로 전 단어에만 의존한다면 단어 열의 확률은 다음과 같다. 이러한 모형을 바이그램 모형 또는 마코프 모형(Markov Model)이라고 한다.

![Bigram](2.png)

## N그램 모형이란?
만약 단어의 활용이 바로 전  n−1 개의 단어에만 의존한다면 단어 열의 확률은 다음과 같다. 이러한 모형을 N그램 모형이라고 한다.

![Ngram](3.png)